# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 71 new papers today
          29 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/28 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.04896


extracting tarball to tmp_2210.04896...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.04902


extracting tarball to tmp_2210.04902...

...

extracting tarball to tmp_2210.05563...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.05595


extracting tarball to tmp_2210.05595...

 done.


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure Model_C_mixture_edited.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.05505-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.05505) | **Quasar and galaxy classification using Gaia EDR3 and CatWise2020**  |
|| <mark>Arvind C.N. Hughes</mark>, <mark>Coryn A.L. Bailer-Jones</mark>, <mark>Sara Jamal</mark> |
|*Appeared on*| *2022-10-12*|
|*Comments*| *21 pages, 23 figures, Accepted for publication in A&A*|
|**Abstract**| In this work, we assess the combined use of Gaia photometry and astrometry with infrared data from CatWISE in improving the identification of extragalactic sources compared to the classification obtained using Gaia data. We evaluate different input feature configurations and prior functions, with the aim of presenting a classification methodology integrating prior knowledge stemming from realistic class distributions in the universe. In our work, we compare different classifiers, namely Gaussian Mixture Models (GMMs), XGBoost and CatBoost, and classify sources into three classes - star, quasar, and galaxy, with the target quasar and galaxy class labels obtained from SDSS16 and the star label from Gaia EDR3. In our approach, we adjust the posterior probabilities to reflect the intrinsic distribution of extragalactic sources in the universe via a prior function. We introduce two priors, a global prior reflecting the overall rarity of quasars and galaxies, and a mixed prior that incorporates in addition the distribution of the these sources as a function of Galactic latitude and magnitude. Our best classification performances, in terms of completeness and purity of the galaxy and quasar classes, are achieved using the mixed prior for sources at high latitudes and in the magnitude range G = 18.5 to 19.5. We apply our identified best-performing classifier to three application datasets from Gaia DR3, and find that the global prior is more conservative in what it considers to be a quasar or a galaxy compared to the mixed prior. In particular, when applied to the pure quasar and galaxy candidates samples, we attain a purity of 97% for quasars and 99.9% for galaxies using the global prior, and purities of 96% and 99% respectively using the mixed prior. We conclude our work by discussing the importance of applying adjusted priors portraying realistic class distributions in the universe. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.05595-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.05595) | **The Exoplanet Radius Valley from Gas-driven Planet Migration and  Breaking of Resonant Chains**  |
|| Andre Izidoro, et al. -- incl., <mark>Christian Zimmermann</mark>, <mark>Bertram Bitsch</mark> |
|*Appeared on*| *2022-10-12*|
|*Comments*| *Accepted for publicationn in ApJL*|
|**Abstract**| The size frequency distribution of exoplanet radii between 1 and 4$R_{\oplus}$ is bimodal with peaks at $\sim$1.4 $R_{\oplus}$ and $\sim$2.4 $R_{\oplus}$, and a valley at $\sim$1.8$R_{\oplus}$. This radius valley separates two classes of planets -- usually referred to as "super-Earths" and "mini-Neptunes" -- and its origin remains debated. One model proposes that super-Earths are the outcome of photo-evaporation or core-powered mass-loss stripping the primordial atmospheres of the mini-Neptunes. A contrasting model interprets the radius valley as a dichotomy in the bulk compositions, where super-Earths are rocky planets and mini-Neptunes are water-ice rich worlds. In this work, we test whether the migration model is consistent with the radius valley and how it distinguishes these views. In the migration model, planets migrate towards the disk inner edge forming a chain of planets locked in resonant configurations. After the gas disk dispersal, orbital instabilities "break the chains" and promote late collisions. This model broadly matches the period-ratio and planet-multiplicity distributions of Kepler planets, and accounts for resonant chains such as TRAPPIST-1, Kepler-223, and TOI-178. Here, by combining the outcome of planet formation simulations with compositional mass-radius relationships, and assuming complete loss of primordial H-rich atmospheres in late giant-impacts, we show that the migration model accounts for the exoplanet radius valley and the intra-system uniformity ("peas-in-a-pod") of Kepler planets. Our results suggest that planets with sizes of $\sim$1.4 $R_{\oplus}$ are mostly rocky, whereas those with sizes of $\sim$2.4 $R_{\oplus}$ are mostly water-ice rich worlds. Our results do not support an exclusively rocky composition for the cores of mini-Neptunes. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.05161-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.05161) | **Spectroscopic follow-up of statistically selected extremely metal-poor  star candidates from GALAH DR3**  |
|| G. S. Da Costa, et al. -- incl., <mark>Arvind C. N. Hughes</mark>, <mark>Lee R. Spitler</mark> |
|*Appeared on*| *2022-10-12*|
|*Comments*| *submitted to MNRAS; 7 pages, 5 figures*|
|**Abstract**| The advent of large-scale stellar spectroscopic surveys naturally leads to the implementation of machine learning techniques to isolate, for example, small sub-samples of potentially interesting stars from the full data set. A recent example is the application of the t-SNE statistical method to $\sim$600,000 stellar spectra from the GALAH survey in order to identify a sample of candidate extremely metal-poor (EMP, [Fe/H] $\leq$ -3) stars. We report the outcome of low-resolution spectroscopic follow-up of 83 GALAH EMP candidates that lack any previous metallicity estimates. Overall, the statistical selection is found to be efficient ($\sim$one-third of the candidates have [Fe/H] $\leq$ -2.75) with low contamination ($<$10% have [Fe/H] $>$ -2), and with a metallicity distribution function that is consistent with previous work. Five stars are found to have [Fe/H] $\leq$ -3.0, one of which is a main sequence turnoff star. Two other stars are revealed as likely CEMP-$s$ stars and a known carbon star is re-discovered. The results indicate that the statistical selection approach employed is valid, and therefore it can be applied to forthcoming even larger stellar spectroscopic surveys with the expectation of similar positive outcomes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.05450-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.05450) | **Forecast of Cosmological Constraints with Type Ia Supernovae from the  Chinese Space Station Telescope**  |
|| <mark>Shi-Yu Li</mark>, et al. -- incl., <mark>Yun-Long Li</mark>, <mark>Tianmeng Zhang</mark> |
|*Appeared on*| *2022-10-12*|
|*Comments*| *10 pages, 6 figures, 2 tables, accepted for publication in SCIENCE CHINA Physics, Mechanics & Astronomy*|
|**Abstract**| The 2-m aperture Chinese Space Station Telescope (CSST), which observes at wavelengths ranging from 255 to 1000 nm, is expected to start science operations in 2024. An ultra-deep field observation program covering approximately 10 square degrees is proposed with supernovae (SNe) and other transients as one of its primary science drivers. This paper presents the simulated detection results of type Ia supernovae (SNe Ia) and explores the impact of new datasets on the determinations of cosmological parameters. The simulated observations are conducted with an exposure time of 150 s and cadences of 10, 20, and 30 days. The survey mode covering a total of 80 observations but with a random cadence in the range of 4 to 14 days is also explored. Our simulation results indicate that the CSST can detect up to $\sim 1800$ SNe Ia at z $<$ 1.3. The simulated SNe Ia are then used to constrain the cosmological parameters. The constraint on $\Omega_m$ can be improved by 37.5% using the 10-day cadence sample in comparison with the Pantheon sample. A deeper measurement simulation with a 300 s exposure time together with the Pantheon sample improves the current constraints on $\Omega_m$ by 58.3% and $\omega$ by 47.7%. Taking future ground-based SNe Ia surveys into consideration, the constraints on $\omega$ can be improved by 59.1%. The CSST ultra-deep field observation program is expected to discover large amounts of SNe Ia over a broad redshift span and enhance our understanding of the nature of dark energy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.05486-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.05486) | **Pulsating H-deficient WDs and pre-WDs observed with TESS: V. Discovery  of two new DBV pulsators, WD J152738.4-450207.4 and WD 1708-871, and  asteroseismology of the already known DBV stars PG 1351+489, EC 20058-5234,  and EC 04207-4748**  |
|| Alejandro H. Córsico, et al. -- incl., <mark>Leandro G. Althaus</mark>, <mark>J. J. Hermes</mark>, <mark>Nicola P. Gentile Fusillo</mark> |
|*Appeared on*| *2022-10-12*|
|*Comments*| *22 pages, 27 figures, 21 tables. To be published in Astronomy & Astrophysics*|
|**Abstract**| The {\sl TESS} space mission has recently demonstrated its great potential to discover new pulsating white dwarf and pre-white dwarf stars, and to detect periodicities with high precision in already known white-dwarf pulsators. We report the discovery of two new pulsating He-rich atmosphere white dwarfs (DBVs) and present a detailed asteroseismological analysis of three already known DBV stars employing observations collected by the {\sl TESS} mission along with ground-based data. We extracted frequencies from the {\sl TESS} light curves of these DBV stars using a standard pre-whitening procedure to derive the potential pulsation frequencies. All the oscillation frequencies that we found are associated with $g$-mode pulsations with periods spanning from $\sim 190$ s to $\sim 936$ s. We find hints of rotation from frequency triplets in some of the targets, including the two new DBVs. For three targets, we find constant period spacings, which allowed us to infer their stellar masses and constrain the harmonic degree $\ell$ of the modes. We also performed period-to-period fit analyses and found an asteroseismological model for three targets, with stellar masses generally compatible with the spectroscopic masses. Obtaining seismological models allowed us to estimate the seismological distances and compare them with the precise astrometric distances measured with {\it Gaia}. We find a good agreement between the seismic and the astrometric distances for three stars (PG~1351+489, EC~20058$-$5234, and EC~04207$-$4748), although for the other two stars (WD~J152738.4$-$50207 and WD~1708$-$871), the discrepancies are substantial. The high-quality data from the {\sl TESS} mission continue to provide important clues to determine the internal structure of pulsating pre-white dwarf and white dwarf stars through the tools of asteroseismology. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.05563-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.05563) | **Searching Lorentz invariance violation from cosmic photon attenuation**  |
|| <mark>Hao Li</mark>, Bo-Qiang Ma |
|*Appeared on*| *2022-10-12*|
|*Comments*| *10 latex pages, 7 figures*|
|**Abstract**| Lorentz invariance violation~(LIV) can change the threshold behavior predicted by special relativity and cause threshold anomalies which affect the propagation of cosmic photons. In this work, we focus on the threshold anomaly effect on cosmic photon attenuations by extragalactic background light~(EBL) and discuss how to identify LIV from observations of very high energy~(VHE) photons propagated from long distance in the universe. We point out that the Large High Altitude Air Shower Observatory~(LHAASO), one of the most sensitive gamma-ray detector arrays currently operating at TeV and PeV energies, is an ideal facility for performing such LIV searching. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.04922-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.04922) | **B-fields do not suppress star formation in low metallicity dwarf  galaxies**  |
|| David J. Whitworth, et al. -- incl., <mark>Rowan J. Smith</mark> |
|*Appeared on*| *2022-10-12*|
|*Comments*| *19 pages, 19 figures, comments welcome*|
|**Abstract**| Many studies have looked at the impact of magnetic fields on star formation in molecular clouds and Milky Way like galaxies, concluding that the field suppresses star formation. However, most of these studies are based on fully developed fields that have reached the saturation level, with little work on investigating how the growth phase of a primordial field affects star formation in low metallicity environments. In this paper, we investigate the impact of the growth phase of a primordial field on low metallicity dwarf galaxies. We perform high-resolution AREPO simulations of 5 isolated dwarf galaxies. Two models are hydrodynamical, two start with a primordial B-field of 10^-6 micro G, and one with a saturated B-field of 10^-2 micro G. All models include a non-equilibrium, time-dependent chemical network that includes the effects of gas shielding from the ambient UV field. Sink particles form directly from the gravitational collapse of gas and are treated as star-forming clumps that can accrete gas. We vary the metallicity, UV-field, and cosmic ray ionization rate between 0.01 and 0.10 of solar values. We find that the magnetic field has little impact on the star formation rate, which is in tension with previously published results. We show that an increase in the mass fractions of both molecular hydrogen and cold gas, along with changes in the perpendicular gas velocity dispersion's and the B-field acting in the weak-field model overcomes the expected suppression in star formation. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.05428-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.05428) | **Lithospheric loading model for large impact basin where mantle plug  presents**  |
|| Qingyun Deng, et al. -- incl., <mark>Zhen Zhong</mark>, <mark>Fei Li</mark> |
|*Appeared on*| *2022-10-12*|
|*Comments*| **|
|**Abstract**| Lithosphere is the outer rigid part of terrestrial body, usually consisting of the crust and part of the mantle. Characterizing the physical properties of lithosphere is critical in the investigation to its evolution history. Through the modeling of mass-related loads within lithosphere, physical parameters such as elastic thickness of lithosphere can be inferred by gravity and topography data. At impact basin region, however, the low correlation between topography and gravity makes this model inapplicable. In this work, we proposed a loading model incorporated with the mantle uplift structure commonly formed at impact basin. The resulting deflection caused by this mantle uplift structure is also modeled in the governing equation of the thin elastic shell. Gravity anomaly of the deflected lithosphere is calculated at the surface and the crustal-mantle boundary, then the theoretical gravity admittance and correlation can be compared with observed data. The application of the mantle loading model at four large impact basins on Mars show better fit to the observed admittance and correlation compared with loading model that without this initial mantle plug. Our work suggests that proper modeling of impact-induced load and its resulting deflection is the key to the understanding of physical properties of planetary lithosphere at basin region. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.05505.md
    + _build/html/tmp_2210.05505/./Plots/Figure2-any_all_featurehist_UWE_w1-w2.png
    + _build/html/tmp_2210.05505/./Plots/Figure3-col_col_diagrams.png
    + _build/html/tmp_2210.05505/./Plots/Figure6-3class_quasar_purity_mixedprior.png
exported in  _build/html/2210.05595.md
    + _build/html/tmp_2210.05595/./P_R_modela_endsim.png
    + _build/html/tmp_2210.05595/./P_R_modelb_endsim.png
    + _build/html/tmp_2210.05595/./P_R_modelc_endsim.png
    + _build/html/tmp_2210.05595/./P_Rm_modela_endsim.png
    + _build/html/tmp_2210.05595/./P_Rm_modelb_endsim.png
    + _build/html/tmp_2210.05595/./P_Rm_modelc_endsim.png
    + _build/html/tmp_2210.05595/./P_Rw_modela_endsim.png
    + _build/html/tmp_2210.05595/./P_Rw_modelb_endsim.png
    + _build/html/tmp_2210.05595/./P_Rw_modelc_endsim.png
    + _build/html/tmp_2210.05595/./Rhist_modela_noobs.png
    + _build/html/tmp_2210.05595/./Rratio_modela_noobs.png
    + _build/html/tmp_2210.05595/./Rhist_modelb_

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\nodata}{ ~\cdots~ }$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\nodata$}{ ~\cdots~ }$</div>



<div id="title">

# Quasar and galaxy classification using Gaia EDR3 and CatWise2020

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.05505-b31b1b.svg)](https://arxiv.org/abs/2210.05505)<mark>Appeared on: 2022-10-12</mark> - _21 pages, 23 figures, Accepted for publication in A&A_

</div>
<div id="authors">

<mark>Arvind C.N. Hughes</mark>, <mark>Coryn A.L. Bailer-Jones</mark>, <mark>Sara Jamal</mark>

</div>
<div id="abstract">

**Abstract:** In this work, we assess the combined use of Gaia photometry and astrometry with infrared data from CatWISE in improving the identification of extragalactic sources compared to the classification obtained using Gaia data. Here we perform a comprehensive study in which we assess different input feature configurations and prior functions to identify extragalactic sources in Gaia, with the aim of presenting a classification methodology integrating prior knowledge stemming from realistic class distributions in the universe.In our work, we compare different classifiers, namely Gaussian Mixture Models (GMMs) and the boosted decision trees, XGBoost and CatBoost, in a supervised approach, and classify sources into three classes - star, quasar, and galaxy, with the target quasar and galaxy class labels obtained from the Sloan Digital Sky Survey Data Release 16 (SDSS16) and the star label from Gaia EDR3.In our approach, we adjust the posterior probabilities to reflect the intrinsic distribution of extragalactic sources in the universe via a prior function. In particular, we introduce two priors, a global prior reflecting the overall rarity of quasars and galaxies, and a mixed prior that incorporates in addition the distribution of the extragalactic sources as a function of Galactic latitude and magnitude.   Our best classification performances, in terms of completeness and purity of the extragalactic classes, namely the galaxy and quasar classes, are achieved using the mixed prior for sources at high latitudes and in the magnitude range G = 18.5 to 19.5.   We apply our identified best-performing classifier to three application datasets from Gaia Data Release 3 (GDR3), and find that the global prior is more conservative in what it considers to be a quasar or a galaxy compared to the mixed prior.   In particular, when applied to the quasar and galaxy candidates tables from GDR3, the classifier using a global prior achieves purities of 55\%for quasars and 93\%for galaxies, and purities of 59\%and 91\%respectively using the mixed prior.   When compared to the  performances obtained on the GDR3 pure quasar and galaxy candidates samples, we reach a higher level of purity, 97\%for quasars and 99.9\%for galaxies using the global prior, and purities of 96\%and 99\%respectively using the mixed prior.   When refining the GDR3 candidates tables via a cross-match with SDSS DR16 confirmed quasars and galaxies, the classifier reaches purities of 99.8\%for quasars and 99.9\%for galaxies using a global prior, and 99.9\%and 99.9\%using the mixed prior.   We conclude our work by discussing the importance of applying adjusted priors portraying realistic class distributions in the universe and the effect of introducing infrared data as ancillary inputs in the identification of extragalactic sources.

</div>

<div id="div_fig1">

<img src="tmp_2210.05505/./Plots/Figure2-any_all_featurehist_UWE_w1-w2.png" alt="Fig11" width="100%"/>

**Figure 11. -** Distribution of the features from the training dataset, coloured according to their true classes. Black: stars, blue: quasars, orange: galaxies. Each distribution is separately normalised and the $sin b$ has been randomised for quasars and galaxies (constant probability
per unit sky area). (*fig:Data-Feature_Hists*)

</div>
<div id="div_fig2">

<img src="tmp_2210.05505/./Plots/Figure3-col_col_diagrams.png" alt="Fig2" width="100%"/>

**Figure 2. -** Colour- magnitude diagram (\textit{top}) and two colour-colour (\textit{middle \& bottom}) diagrams highlighting the distribution of each class, with contours designating the density on a linear-scale for a random sample of 10,000 observations. The color black corresponds to stars, blue for quasars and gold for galaxies. Distinct aggregates can be identified for each class, although a significant interclass overlap still occurs. (*fig:Data-col_col*)

</div>
<div id="div_fig3">

<img src="tmp_2210.05505/./Plots/Figure6-3class_quasar_purity_mixedprior.png" alt="Fig13" width="100%"/>

**Figure 13. -** Imbalanced Classifier Mixed Prior: Purity evaluated for the quasar class in the test set from predictions obtained by the best performing models, i.e. XGBoost, trained on the \textit{Feature Set 1}(left panel) and the \textit{Feature Set 4}(right panel).
Top-panels show the classification performances modified by the adjustment factor. The near unit purity at low latitudes in the right panels is not meaningful as there a very few objects as shown in Fig. \ref{fig:Data-mixedpriors} (*fig:Results-3class_quasar_purity_mixedprior*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\ai}[1]{\textcolor{red}{[#1]}}$
$\newcommand{\thefigure}{A\arabic{figure}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\ai$}[1]{\textcolor{red}{[#1]}}$
$\newcommand{$\thefigure$}{A\arabic{figure}}$</div>



<div id="title">

# The Exoplanet Radius Valley from Gas-driven Planet Migration and Breaking of Resonant Chains

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.05595-b31b1b.svg)](https://arxiv.org/abs/2210.05595)<mark>Appeared on: 2022-10-12</mark> - _Accepted for publicationn in ApJL_

</div>
<div id="authors">

André Izidoro, et al. -- incl., <mark>Christian Zimmermann</mark>, <mark>Bertram Bitsch</mark>

</div>
<div id="abstract">

**Abstract:** The size frequency distribution of exoplanet radii between 1 and 4$R_{\oplus}$is bimodal with peaks  at$\sim$1.4$R_{\oplus}$and$\sim$2.4$R_{\oplus}$, and a valley at$\sim$1.8$R_{\oplus}$. This radius valley separates two classes of planets -- usually referred to as "super-Earths"  and "mini-Neptunes" -- and its origin remains debated. One model proposes that super-Earths are the outcome of photo-evaporation or core-powered mass-loss  stripping the primordial atmospheres of the mini-Neptunes. A contrasting model interprets the radius valley as a dichotomy in the bulk compositions, where super-Earths are  rocky planets and mini-Neptunes are water-ice rich worlds. In this work, we test whether the migration model is consistent with the radius valley and how it distinguishes these views. In the migration model, planets migrate towards the disk inner edge forming a chain of planets locked in resonant configurations. After the gas disk dispersal, orbital instabilities  "break the chains" and promote  late collisions. This model broadly matches the  period-ratio and planet-multiplicity distributions of Kepler planets, and  accounts for resonant chains such as TRAPPIST-1, Kepler-223, and TOI-178. Here, by combining the outcome of planet formation simulations with compositional mass-radius relationships, and assuming complete loss of primordial H-rich atmospheres in late giant-impacts, we show that the migration model accounts for the exoplanet radius valley and the intra-system uniformity ("peas-in-a-pod") of Kepler planets. Our results suggest that planets with sizes of$\sim$1.4$R_{\oplus}$are mostly rocky, whereas  those with sizes of$\sim$2.4$R_{\oplus}$are mostly water-ice rich worlds. Our results do not support an exclusively rocky composition for the cores of mini-Neptunes.

</div>

<div id="div_fig1">

<img src="tmp_2210.05595/./P_R_modela_endsim.png" alt="Fig8.1" width="11%"/><img src="tmp_2210.05595/./P_R_modelb_endsim.png" alt="Fig8.2" width="11%"/><img src="tmp_2210.05595/./P_R_modelc_endsim.png" alt="Fig8.3" width="11%"/><img src="tmp_2210.05595/./P_Rm_modela_endsim.png" alt="Fig8.4" width="11%"/><img src="tmp_2210.05595/./P_Rm_modelb_endsim.png" alt="Fig8.5" width="11%"/><img src="tmp_2210.05595/./P_Rm_modelc_endsim.png" alt="Fig8.6" width="11%"/><img src="tmp_2210.05595/./P_Rw_modela_endsim.png" alt="Fig8.7" width="11%"/><img src="tmp_2210.05595/./P_Rw_modelb_endsim.png" alt="Fig8.8" width="11%"/><img src="tmp_2210.05595/./P_Rw_modelc_endsim.png" alt="Fig8.9" width="11%"/>

**Figure 8. -** Architecture of planetary systems at the end of our simulations, i.e., after dynamical instabilities have taken place. The horizontal axis shows orbital period, and the y-axis planetary radius as calculate from the mass radius relationships of \cite{zengetal19}.  Planets are shown as individual dots. We only show planets with orbital periods shorter than 100 days. In all these models, more than $\sim$95\% of the resonant chains became unstable after gas disk dispersal. Models A, B, and C are shown from left to right. Points are color-coded according to the planets'
number of late giant impacts (top row of panels), masses (middle row of panels), and ice-mass fractions (right column). We neglect the effects of photo-evaporation and/or other subsequent atmospheric mass-loss in all these cases (see Appendix for simulations where the effects of photoevaporation are included). The dashed lines in the bottom panels show the exoplanet radius valley slope, calculated  as $R =10^{-0.11{\rm log}_{10}(P)+0.4}$ \citep{vaneylenetal18,guptaetal19}. The green background-contours show the kernel density distribution of the planets in our simulations (dots of the figure). (*fig:after_instabilities*)

</div>
<div id="div_fig2">

<img src="tmp_2210.05595/./Rhist_modela_noobs.png" alt="Fig9.1" width="16%"/><img src="tmp_2210.05595/./Rratio_modela_noobs.png" alt="Fig9.2" width="16%"/><img src="tmp_2210.05595/./Rhist_modelb_noobs.png" alt="Fig9.3" width="16%"/><img src="tmp_2210.05595/./Rratio_modelb_noobs.png" alt="Fig9.4" width="16%"/><img src="tmp_2210.05595/./Rhist_modelc_noobs.png" alt="Fig9.5" width="16%"/><img src="tmp_2210.05595/./Rratio_modelc_noobs.png" alt="Fig9.6" width="16%"/>

**Figure 9. -** {\bf Left:}  Planet radius distribution. {\bf Right:} Planet size-ratio  distribution of adjacent planet pairs. These distributions are computed after dynamical instabilities and the breaking of the resonant chains. Exoplanets are shown in gray. Blue shows the outcome of our planet formation simulations. Red shows the outcome of our planet formation simulations including the effects of photo-evaporation. From top-to-bottom, the panel-rows show model A,  B, and  C. For all models, we use the mass-radius relationship of \cite{zengetal19} assuming an uncertainty of 7\% in size and an initial atmosphere-to-core mass ratio of 0.3\%. We build these histograms accounting for uncertainties in radius by generating the possible radius of each planet 100 times. (*fig:distributions*)

</div>
<div id="div_fig3">

<img src="tmp_2210.05595/./Model_C_mixture_edited.png.png" alt="Fig11" width="100%"/>

**Figure 11. -** Final planetary systems produced in Model-C.  Each line shows one planetary systems. The color-coding indicates the planet composition.  A black-ring around the dot indicates the presence of  atmosphere, and the full-size of the dot scales with its radius. When determining the final planet sizes, we include atmospheric stripping by photo-evaporation after the giant impact phase. Planets atmospheric masses, when present, are assumed to correspond to 0.3\% of the core mass. (*fig:final_systems*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

61  publications in the last 7 days.
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers